In [ ]:
from datasets import load_dataset

# Load the MedQA dataset (US version)
dataset = load_dataset("VodLM/medqa", "tw")

# Check available splits
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['idx', 'uid', 'question', 'metamap', 'target', 'answers'],
        num_rows: 11298
    })
    validation: Dataset({
        features: ['idx', 'uid', 'question', 'metamap', 'target', 'answers'],
        num_rows: 1412
    })
    test: Dataset({
        features: ['idx', 'uid', 'question', 'metamap', 'target', 'answers'],
        num_rows: 1413
    })
})


In [ ]:
train_data = dataset["train"]
print(train_data[0])  # Print the first data point



import pandas as pd

# Convert dataset to pandas DataFrame
train_data = dataset["train"].to_pandas()
test_data = dataset["test"].to_pandas()
validation_data = dataset["validation"].to_pandas()




{'idx': 0, 'uid': 'train-0', 'question': 'After the reaction physiology Which is not bedridden patients not moving (immobilization)?', 'metamap': '', 'target': 2, 'answers': ['Muscle atrophy', 'Weakness', 'Ligamentous laxity, increased ductility', 'Poor motor coordination']}


In [ ]:
train_data.head()

,idx,uid,question,metamap,target,answers
0,0,train-0,After the reaction physiology Which is not bed...,,2,"[Muscle atrophy, Weakness, Ligamentous laxity,..."
1,1,train-1,"Humans, vitamin D3 is converted to 25-hydroxyc...",,2,"[skin, kidney, liver, parathyroid gland]"
2,2,train-2,Esotropia eye when doing alternate cover test ...,,1,"[In the exhibition (adduction), Abduction (Abd..."
3,3,train-3,"Suppose there is a popular city Influenza A, 1...",,2,"[0.1, 0.002, 0.2, 0.005]"
4,4,train-4,Healthcare nuclear emergency physician is noti...,,2,[To pre-hospital patient's death does not requ...


In [ ]:
# with pd.ExcelWriter("dataset.xlsx", engine="openpyxl") as writer:
#     train_data.to_excel(writer, sheet_name="Train", index=False)
#     test_data.to_excel(writer, sheet_name="Test", index=False)
#     validation_data.to_excel(writer, sheet_name="Validation", index=False)

# print("Excel file 'dataset.xlsx' created with 3 sheets: Train, Test, and Validation.")

In [ ]:
!pip install sacremoses

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# 1. Configure model & device
model_name = "microsoft/BioGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 2. Define helpers
LETTER_MAP = ["A", "B", "C", "D"]

# Your one few‑shot example
few_shot_example = {
    "question": "Which organelle is known as the “powerhouse of the cell”?",
    "options": ["Lysosome", "Mitochondrion", "Golgi apparatus", "Endoplasmic reticulum"],
    "answer": "B"
}

def create_prompt(question, options, few_shot_examples=None):
    prompt = ""
    if few_shot_examples:
        for ex in few_shot_examples:
            prompt += (
                f"Question: {ex['question']}\n"
                "Options:\n"
                f"A: {ex['options'][0]}\n"
                f"B: {ex['options'][1]}\n"
                f"C: {ex['options'][2]}\n"
                f"D: {ex['options'][3]}\n"
                f"Answer: {ex['answer']}\n\n"
            )
    prompt += (
        f"Question: {question}\n"
        "Options:\n"
        f"A: {options[0]}\n"
        f"B: {options[1]}\n"
        f"C: {options[2]}\n"
        f"D: {options[3]}\n"
        "Answer:"
    )
    return prompt

# 3. Run inference over validation_data
predicted = []
gold = []

for _, row in validation_data.iterrows():
    q = row["question"]
    opts = row["answers"]      # list of 4 strings
    true_idx = int(row["target"])
    gold_label = LETTER_MAP[true_idx]
    gold.append(gold_label)

    prompt = create_prompt(q, opts, few_shot_examples=[few_shot_example])
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # generate only a few tokens beyond the prompt
    out = model.generate(
        **inputs,
        max_new_tokens=5,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    # decode only the newly generated portion
    gen = tokenizer.decode(out[0, inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    # take first non‑empty token, uppercase and strip punctuation
    pred = gen.strip().split()[0].upper().rstrip(".:")
    # ensure it’s one of A/B/C/D
    if pred not in LETTER_MAP:
        pred = "A"  # fallback or you could mark as incorrect
    predicted.append(pred)

# 4. Compute and print metrics
acc = accuracy_score(gold, predicted)
print(f"Validation Accuracy: {acc:.4f}\n")
print("Classification Report:")
print(classification_report(gold, predicted, labels=LETTER_MAP))


Validation Accuracy: 0.2217

Classification Report:
              precision    recall  f1-score   support

           A       0.20      0.55      0.29       286
           B       0.26      0.42      0.32       365
           C       0.00      0.00      0.00       410
           D       0.50      0.00      0.01       351

    accuracy                           0.22      1412
   macro avg       0.24      0.24      0.15      1412
weighted avg       0.23      0.22      0.14      1412

